# Hydrogeodesy: Monitoring surface waters from space
### Exercise 2: Comparison of different inland altimetry products

Daniel Scherer, DGFI-TUM  
Wintersemester 2022/23

**Contents**
1. Reading NetCDF formatted data
2. Data processing
3. Quality assessment

**Study Area: Lake Erie (USA)**  
![AOI](aoi.Png)  
*Figure 1: Lake Erie with intersecting satellite altimetry passes. Red: Topex/Poseidon and Jason-1/-2 missions. Blue: ERS-2 and Envisat missions*  

Water Level Variation: 173 – 176 m

Following data is used in this exercise:  

| Dataset | Institution | Epoche | Samples | Filename |
| --- | --- | --- | --- | --- |
| [In-Situ](https://tidesandcurrents.noaa.gov/waterlevels.html?id=9063053) | NOAA | 1992-2015 | 8489 (daily) | gauge_33.nc |
| [DAHITI](https://dahiti.dgfi.tum.de/en/6/water-level-altimetry/) | DGFI-TUM | 1992-2017 | 2289 | dahiti_6.nc |
| [Hydroweb](https://hydroweb.theia-land.fr/hydroweb/view/L_erie?lang=en) | LEGOS | 1992-2011 | 222 | hydroweb_1560.nc |
| [River&Lake](http://altimetry.esa.int/riverlake/shared/main.html) | ESA | 2002-2010 | 81 | Esa_778.nc |
| [GRLM](https://ipad.fas.usda.gov/cropexplorer/global_reservoir/gr_regional_chart.aspx?regionid=us&reservoir_name=Erie) | USDA | 1992-2014 | 752 | grlm_75.nc |


#### 1. Reading NetCDF formatted data
Use the netcdf4 package to read NetCDF data in python.  
e.g. the DAHITI data:

In [ ]:
import netCDF4 as nc
dset = nc.Dataset('data/dahiti_6.nc')

You can get information about the data set by just printing it:

In [ ]:
print(dset)

And inspect the variables and dimensions by:

In [ ]:
print(dset.variables)

You can access a variable as shown, it will return a numpy array:

In [ ]:
heights_dahiti = dset['height'][:]
print(type(heights_dahiti))
print(heights_dahiti)

It is good practice to eventually close the dataset to prevent memory leaks and permission issues:

In [ ]:
dset.close()

Extract the water level data from the files in the *data* folder.  
To make the following steps easier, we create a ***[Pandas Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html)*** object for each source.  
The data of the series should be the water level and the index should be the respective julian days.

In [ ]:
import pandas as pd

dahiti = pd.Series(data = ..., index = ...)

esa = ...

gauge = ...

grlm = ...
# Somehow the GRLM data contains duplicated entries but we keep only the first:
grlm = grlm[~grlm.index.duplicated(keep='first')]

hw = ...

Plot the Data:

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
dahiti.plot(ax=ax,color="blue",legend=True,label='DAHITI')
esa.plot(ax=ax,color="green",legend=True,label='ESA')
gauge.plot(ax=ax,color="black",legend=True,label='Gauge')
grlm.plot(ax=ax,color="red",legend=True,label='GRLM')
hw.plot(ax=ax,color="orange",legend=True,label='HydroWeb')
plt.ylabel("Water Level [m]")
plt.xlabel("Julian Days")
plt.show()

As you can see, there are significant offsets between the time series, as they relate to different references.  
We will fix this in the next steps.

#### 2. Data Processing

Prepare the data to compare each altimetry dataset (dahiti, hydroweb, esa, grlm) with the in-situ gauge data.

There may be an offset between both datasets, e.g. because of different vertical datums, which should be removed.

We do ot need to find intersecting days of observations in advance, because we use the pandas series objects.  
The index (julian days) is automatically compared during the operations:

In [ ]:
offset_dahiti = (gauge - dahiti).median()
dahiti += offset_dahiti

Compute and apply the offset for each dataset and plot the data as above:

In [ ]:
offset_hw = ...
hw += ...
offset_esa = ...
esa += ...
offset_grlm = ...
grlm += ...

fig, ax = plt.subplots()
dahiti.plot(ax=ax,color="blue",legend=True,label='DAHITI')
esa.plot(ax=ax,color="green",legend=True,label='ESA')
gauge.plot(ax=ax,color="black",legend=True,label='Gauge')
grlm.plot(ax=ax,color="red",legend=True,label='GRLM')
hw.plot(ax=ax,color="orange",legend=True,label='HydroWeb')
plt.ylabel("Water Level [m]")
plt.xlabel("Julian Days")
plt.show()

#### Quality Assessment

Build functions to calculate the **Root Mean Square Error (RMSE), Nash-Sutcliff Efficiency (NSE), and squared Pearson's correlation coefficient (R²)** between both datasets.

The formulas are given in the lecture slides "Satellite Altimetry (Part 2)" pp. 47ff.

You can either use pandas buildin functions (*i.e. (series_a - series_b).mean()*) or numpy on the intersection of both series:

    idx = series_a.index.intersection(series_b.index)
    array_a, array_b = series_a[idx].to_numpy(), series_b[idx].to_numpy()
    
Useful numpy functions are:
- np.power(*base, exponent*)
- np.sum(*values*)
- np.cov(*values_a, values_b*)
- np.std(*values*)
- np.mean(*values*)

In [ ]:
import numpy as np

def rmse(a, b):
    return ...

def corr(a, b):
    return ...

def nse(a,b):
    return ...

Apply the functions to compare each dataset with insitu.  

In [ ]:
rmse_dahiti = rmse(dahiti,gauge)
corr_dahiti = corr(dahiti,gauge)
nse_dahiti = nse(dahiti,gauge)

rmse_hw = rmse(hw,gauge)
corr_hw = corr(hw,gauge)
nse_hw = nse(hw,gauge)

rmse_esa = rmse(esa,gauge)
corr_esa = corr(esa,gauge)
nse_esa = nse(esa,gauge)

rmse_grlm = rmse(grlm,gauge)
corr_grlm = corr(grlm,gauge)
nse_grlm = nse(grlm,gauge)

#### 3. Quality Assessment
Display a summary of the quality measures.
Interpret and discuss the results:

In [ ]:
import pandas as pd
pd.DataFrame.from_dict({
    "Agency": ["DGFI-TUM","LEGOS","ESA","USDA"],
    "Name" : ['DAHITI','Hydroweb',"Rivers & Lakes",'GRLM'],
    "Offset [m]" : np.round([offset_dahiti, offset_hw, offset_esa, offset_grlm],3),
    'RMSE [m]' : np.round([rmse_dahiti, rmse_hw, rmse_esa, rmse_grlm],3),
    'Corr.' : np.round([corr_dahiti, corr_hw, corr_esa, corr_grlm],3),
    'NSE' : np.round([nse_dahiti, nse_hw, nse_esa, nse_grlm],3)
},orient='index')